In [1]:
# Import necessary libraries
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


In [2]:
# Load the dataset (top 10,000 most frequent words)
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
# Function to one-hot encode the sequences into vectors of size 10,000
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.0
    return results

In [4]:
# Vectorize training and test data
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

In [5]:
# Convert labels to float32
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

In [6]:
# Define the deep neural network model
model = Sequential()
model.add(Dense(16, activation='relu', input_shape=(10000,)))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  # Sigmoid for binary output

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
# Compile the model with appropriate loss function for binary classification
model.compile(optimizer=Adam(),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [8]:
# Display model architecture
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 16)             │       160,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 160,305 (626.19 KB)

 Trainable params: 160,305 (626.19 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# Early stopping to avoid overfitting
early_stop = EarlyStopping(monitor='val_loss', patience=3)

In [10]:
# Train the model
history = model.fit(x_train, y_train,
                    epochs=20,
                    batch_size=512,
                    validation_split=0.3,
                    callbacks=[early_stop],
                    verbose=1)

Epoch 1/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 4s 73ms/step - accuracy: 0.6817 - loss: 0.6132 - val_accuracy: 0.8692 - val_loss: 0.3577
Epoch 2/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.9023 - loss: 0.2861 - val_accuracy: 0.8887 - val_loss: 0.2779
Epoch 3/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.9389 - loss: 0.1887 - val_accuracy: 0.8900 - val_loss: 0.2721
Epoch 4/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.9575 - loss: 0.1400 - val_accuracy: 0.8880 - val_loss: 0.2850
Epoch 5/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.9686 - loss: 0.1108 - val_accuracy: 0.8839 - val_loss: 0.3079
Epoch 6/20
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.9794 - loss: 0.0857 - val_accuracy: 0.8787 - val_loss: 0.3367


In [11]:
# Evaluate the model on the test data
results = model.evaluate(x_test, y_test)
print(f"\nTest Loss: {results[0]:.4f}, Test Accuracy: {results[1]:.4f}")

782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8708 - loss: 0.3642

Test Loss: 0.3647, Test Accuracy: 0.8707


In [12]:
# Predict probabilities for the first 10 test reviews
predictions = model.predict(x_test[:10])

# Display predicted and actual labels
for i in range(10):
    print(f"Review #{i+1}")
    print(f"Predicted Probability (Positive): {predictions[i][0]:.4f}")
    print(f"Predicted Label: {'Positive' if predictions[i][0] >= 0.5 else 'Negative'}")
    print(f"Actual Label: {'Positive' if y_test[i] == 1 else 'Negative'}\n")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
Review #1
Predicted Probability (Positive): 0.2136
Predicted Label: Negative
Actual Label: Negative

Review #2
Predicted Probability (Positive): 1.0000
Predicted Label: Positive
Actual Label: Positive

Review #3
Predicted Probability (Positive): 0.2567
Predicted Label: Negative
Actual Label: Positive

Review #4
Predicted Probability (Positive): 0.7781
Predicted Label: Positive
Actual Label: Negative

Review #5
Predicted Probability (Positive): 0.9700
Predicted Label: Positive
Actual Label: Positive

Review #6
Predicted Probability (Positive): 0.9216
Predicted Label: Positive
Actual Label: Positive

Review #7
Predicted Probability (Positive): 1.0000
Predicted Label: Positive
Actual Label: Positive

Review #8
Predicted Probability (Positive): 0.0025
Predicted Label: Negative
Actual Label: Negative

Review #9
Predicted Probability (Positive): 0.9957
Predicted Label: Positive
Actual Label: Negative

Review #10
Predicted Probability (Positive): 0.9996
P